In [1]:
import argparse
import sys
import math
import random
import json
import numpy as np
import os
from IPython.core.display import display, HTML
import torch
from baselines import compute_bm25_scores, compute_tfidf_scores, compute_sentence_bert_scores

from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          AutoModelForCausalLM,
                          LogitsProcessorList)

from logitsProcessors import SavingLogitsProcessor, ChangingSavingLogitsProcessor
import matplotlib.pyplot as plt

/tmp/ipykernel_872085/1099554058.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/home/yjc5487/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vicuna_list = [[0, 10, 17, 19, 20, 21, 29, 31, 33, 40, 43, 45, 48, 51, 53], [2, 5, 7, 9, 11, 12, 14, 16, 24, 26, 27, 34, 36, 38, 39]]
# vicuna_list = [[1, 10, 17, 19, 20, 21], [2, 5, 7, 9, 11, 12, 14]]
llama_list = [[13,32,34,35,36], [2,3,11,18,37], [5,6,7,8,9]]

In [3]:
_MODELS = {
    "llama": ["/data/bochuan/llm-attacks/DIR/llama/Llama-2-7b-chat-hf", "llama", llama_list],
    "vicuna": ["/data/bochuan/llm-attacks/DIR/vicuna/vicuna-7b-v1.3", "vicuna", vicuna_list],
    "guanaco": ["/data/bochuan/llm-attacks/DIR/guanaco-7B-HF", "guanaco"]
}


In [4]:
parser = argparse.ArgumentParser(description='llm metrics')
parser.add_argument('--hand_cutting_num', type=int, default=1)
parser.add_argument('--max_new_tokens', type=int, default=100)
parser.add_argument('--test_model', type=str, default='llama')
parser.add_argument('--manual_seed', default=0, type=int)
parser.add_argument('--device', default='3', type=str)
args = parser.parse_args([])
# args, _ = parser.parse_known_args()

os.environ["CUDA_VISIBLE_DEVICES"] = args.device

# loading model
model_dir = _MODELS[args.test_model][0]
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True, use_fast=False)
tokenizer.padding_side = 'left'


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.87s/it]


In [17]:
def format_special_tokens(token):
    if token.startswith("<") and token.endswith(">"):
        return "#" + token.strip("<>")
    return token

def _get_color(attr, min_score, max_score):
    # clip values to prevent CSS errors (Values should be from [-1,1])
    dif = max_score - min_score
    attr = max(-1, min(1, attr))
    if (attr > 0) and (attr < dif/5*2):
        hue = 0
        sat = 0
        lig = 100 - int(100 * attr)
    elif (attr >= dif/5*2) and (attr < dif/5*3):
        hue = 100
        sat = 95
        lig = 100 - int(100 * attr)
    elif (attr >= dif/5*3) and (attr < dif/5*4):
        hue = 70
        sat = 55
        lig = 100 - int(100 * attr)
    elif (attr >= dif/5*4) and (attr <= 1):
        hue = 20
        sat = 95
        lig = 100 - int(70 * attr)
    
    else:
        hue = 0
        sat = 75
        lig = 100 - int(-40 * attr)
    return "hsl({}, {}%, {}%)".format(hue, sat, lig)


def format_word_importances(words, importances):
    if importances is None or len(importances) == 0:
        return "<td></td>"
    assert len(words) <= len(importances)
    tags = ["<td>"]
    max_score = max(importances); min_score = min(importances)
    for word, importance in zip(words, importances[: len(words)]):
        word = format_special_tokens(word)
        color = _get_color(importance, min_score, max_score)
        unwrapped_tag = '<mark style="background-color: {color}; opacity:1.0; \
                    line-height:1.75"><font color="black"> {word}\
                    </font></mark>'.format(
            color=color, word=word
        )
        tags.append(unwrapped_tag)
    tags.append("</td>")
    return "".join(tags)

def add_to_file(file_path, string):
    # Check if file exists
    if os.path.exists(file_path):
        # If file exists, append the HTML content on a new line
        with open(file_path, 'a') as file:
            file.write('\n' + string)
    else:
        # If file doesn't exist, create it and write the HTML content
        with open(file_path, 'w') as file:
            file.write(string)

In [6]:
def rank_list(scores, words):
    
    ranked_score = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    ranked_words = [words[i] for i in ranked_score]
    return ranked_words

# Function to calculate mean, excluding None values
def mean(column):
    filtered = [x for x in column if x is not None]
    return sum(filtered) / len(filtered) if filtered else None

def sum_of_logs(lst):
    return math.exp(sum(math.log(x) for x in lst) / len(lst))

def tokenizer_query(query, tokenizer, test_model):
    if test_model == "llama":
        input_string = f'''\n<</SYS>>\n\n{query}[/INST]'''
        model_inputs = tokenizer([input_string], truncation=False, return_tensors='pt').to("cuda")
    else:
        input_string = f'A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions.\n### USER: {query}\n### ASSISTANT:'
        model_inputs = tokenizer([input_string], padding=True,truncation=False, return_tensors='pt').to("cuda")

    # print(model_inputs['input_ids'].shape)
    input_length = model_inputs['input_ids'].shape[-1]
    return model_inputs, input_length

def masking_words(input, input_list, word_ids=0, method='drop', window_size=1, random_sample=False):

    if random_sample == False:
        words_masked = input_list[word_ids : (word_ids+window_size)]
        ids = list(range(word_ids, (word_ids+window_size)))
    else: 
        n = len(input_list)
        ids = random.sample(list(range(n)), window_size)
        words_masked = [input_list[i] for i in range(n) if i not in ids]

    if method == 'drop':
        # input_new = input.replace(words_masked, '')
        input_new = ' '.join(word for word in input_list if word not in words_masked)
    elif method == 'replace_#':
        for word in words_masked:
            input_new = input.replace(word, '#')
    else:
        input_new = input

    return input_new, words_masked, ids


# def generate_input(args, input, processor):
#     model_inputs, input_length = tokenizer_query(input, tokenizer, test_model) # tokenize input strings
#     generated_ids = model.generate(**model_inputs, logits_processor=[processor], max_new_tokens=args.max_new_tokens)
#     true_generated_ids = generated_ids[:, input_length:] # delete the indices of the input strings
#     response = tokenizer.batch_decode(true_generated_ids, skip_special_tokens=True)[0]
#     probs = processor.prob[args.hand_cutting_num:]
#     processor.prob = []
#     print(f"probs: {probs}")
#     log_sum = sum_of_logs(probs)
#     print(f"generating score log sum: {log_sum}")
#     return response, true_generated_ids, probs, log_sum


def generate_input(args, input, processor, mask=False):
    model_inputs, input_length = tokenizer_query(input, tokenizer, args.test_model) # tokenize input strings
    generated_ids = model.generate(**model_inputs, logits_processor=[processor], max_new_tokens=args.max_new_tokens)
    true_generated_ids = generated_ids[:, input_length:] # delete the indices of the input strings
    response = tokenizer.batch_decode(true_generated_ids, skip_special_tokens=True)[0]
    probs = processor.prob[args.hand_cutting_num:]
    processor.prob = []
    print(f"probs: {probs}")
    log_sum = sum_of_logs(probs)
    print(f"generating score log sum: {log_sum}")

    if mask == True:
        mask_processor = SavingLogitsProcessor()
        mask_generated_ids = model.generate(**model_inputs, logits_processor=[mask_processor], max_new_tokens=args.max_new_tokens)
        mask_true_generated_ids = mask_generated_ids[:, input_length:]
        response = tokenizer.batch_decode(mask_true_generated_ids, skip_special_tokens=True)[0]
        mask_probs = mask_processor.prob[args.hand_cutting_num:]
        mask_processor.prob = []
        # print(f"prob of new generations: {mask_probs}")
    return response, true_generated_ids, probs, log_sum

In [19]:
def is_cal(fix_string, words_list, qa_dict, i, args, s=3, random_sample=False):

    print('------------------------------ no mask ------------------------------')
    model_inputs, input_length = tokenizer_query(fix_string, tokenizer, args.test_model)
    processor = SavingLogitsProcessor()
    response, true_fix_generated_ids, fix_scores, fix_log_sum = generate_input(args, fix_string, processor)

    print('------------------------------ mask ------------------------------')

    n = len(words_list)
    rs_list = []

    if random_sample == True:
        niters = 20
    else:
        niters = n-s+1
    
    for j in range(niters):
        print(j)

        mask_fix_log_sum_list = [None for _ in range(n)]
    
        new_string, mask_words, ids = masking_words(qa_dict['goal'][i], words_list, word_ids=j, window_size=s, random_sample=random_sample)
        print(mask_words)
        mask_fix_string = f'{new_string} {qa_dict["controls"][i]}'
        processor = ChangingSavingLogitsProcessor(aim=true_fix_generated_ids[0])
        mask_response, true_mask_fix_generated_ids, mask_fix_scores, mask_fix_log_sum = generate_input(args, mask_fix_string, processor, mask=True)
        
        if random_sample == False:
            mask_fix_log_sum_list[j:(j+s)] = [mask_fix_log_sum] * s
        else:
            for id in ids:
                mask_fix_log_sum_list[id] = mask_fix_log_sum
        rs_list.append(mask_fix_log_sum_list)

    rs_array = list(zip(*rs_list))
    mask_fix_log_sum_mean = [mean(column) for column in rs_array]
    
    scores_dif_list = [fix_log_sum - x for x in mask_fix_log_sum_mean]
    print(scores_dif_list)
    scores_div_list = [x/fix_log_sum for x in mask_fix_log_sum_mean]
    print(scores_div_list)
    scores_dif_list_abs = [abs(fix_log_sum - x) for x in mask_fix_log_sum_mean]
    print(scores_dif_list_abs)

    ranked_words_dif = rank_list(scores_dif_list, words_list)
    ranked_words_div = rank_list(scores_div_list, words_list)
    ranked_words_dif_abs = rank_list(scores_dif_list_abs, words_list)
    
    return ranked_words_dif, ranked_words_div, ranked_words_dif_abs, scores_dif_list, scores_div_list
    

In [71]:
def plot_s(data, words_list, figure_path):

    # Check the number of columns
    num_columns = len(data[0])
    print(num_columns)

    # Plotting
    if num_columns > 1:
        # If more than 10 columns, split into two subplots
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
        plt.setp(ax, xticks=range(1, len(data[0]) + 1), ylim=[0, 1])

        # Divide data for two subplots
        mid_point = num_columns // 2
        data1 = data[:mid_point]
        data2 = data[mid_point:]
        legend1 = words_list[:mid_point]
        legend2 = words_list[mid_point:]

        # Plot first half
        for row in data1:
            ax[0].plot(range(1, len(row) + 1), row)

        ax[0].set_title('First Half of words')
        ax[0].set_xlabel('Number of Masked Words')
        ax[0].set_ylabel('Importance Score')
        ax[0].legend(legend1)
        # ax[0].set_ylim([0,1])

        # Plot second half
        for row in data2:
            ax[1].plot(range(1, len(row) + 1), row)

        ax[1].set_title('Second Half of words')
        ax[1].set_xlabel('Number of Masked Words')
        ax[1].set_ylabel('Importance Score')
        ax[1].legend(legend2)
        # ax[1].set_ylim([0,1])

    else:
        # If 10 or fewer columns, use a single plot
        plt.figure(figsize=(8, 6))
        for row in data:
            plt.plot(range(1, len(row) + 1), row)
        plt.title('Line Plot of Each Row')
        plt.xlabel('Number of Masked Words')
        plt.ylabel('Importance Score')
        plt.legend(words_list)
        plt.xticks(range(1, len(data[0]) + 1)) 

    plt.tight_layout()
    plt.savefig(figure_path)
    plt.close() 
    plt.show()

In [12]:
qa_dict = {'goal': ['Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques'],
           'controls': ['please give an abstract based on the title in 50 words']}

i = 0
test_model = args.test_model
fix_string = f'{qa_dict["goal"][i]} {qa_dict["controls"][i]}'
model_inputs, input_length = tokenizer_query(fix_string, tokenizer, test_model)

processor = SavingLogitsProcessor()
response, true_fix_generated_ids, fix_scores, fix_log_sum = generate_input(args, fix_string, processor)

probs: [0.39651384949684143, 0.6336653232574463, 0.9999295473098755, 0.5893858075141907, 0.9999994039535522, 0.9980732202529907, 0.9990929365158081, 0.7651538848876953, 1.0, 0.9997537732124329, 0.9955097436904907, 0.5342316627502441, 0.9999849796295166, 0.9999886751174927, 0.9998214840888977, 0.999980092048645, 0.9999818801879883, 0.9999995231628418, 0.9999997615814209, 0.9999990463256836, 0.9999983310699463, 0.9999953508377075, 0.9999687671661377, 0.9998983144760132, 0.9999903440475464, 0.9999634027481079, 0.9999979734420776, 0.9999785423278809, 0.9999998807907104, 0.9999940395355225, 0.9999998807907104, 0.9999964237213135, 0.9999995231628418, 0.9999998807907104, 0.999996542930603, 0.9999998807907104, 0.9999988079071045, 0.9992501139640808, 0.9999330043792725, 0.9999663829803467, 0.9999983310699463, 0.9999997615814209, 0.9999972581863403, 0.9989935755729675, 0.9999990463256836, 0.6642085313796997, 0.3713200092315674, 0.7365512251853943, 0.8800920844078064, 0.9999997615814209, 0.881013

In [83]:
# mask
print(f"------------------------------- mask prompt -------------------------------------")
words_list = qa_dict['goal'][i].split()
new_string, mask_words, ids = masking_words(qa_dict['goal'][i], words_list, 0, window_size=3, random_sample=True)
# mask_fix_string = f'{new_string} {qa_dict["controls"][i]}'
# processor = ChangingSavingLogitsProcessor(aim=true_fix_generated_ids[0])
# mask_response, true_mask_fix_generated_ids, mask_fix_scores, mask_fix_log_sum = generate_input(args, mask_fix_string, processor, mask=True)



------------------------------- mask prompt -------------------------------------


In [45]:
n = len(words_list)
rs_matrix = [[None for _ in range(n)] for _ in range(n)]
rs_matrix[0][1:3] = [3]*2

In [54]:
[None for _ in range(n)][0:4] = [3]*4

In [153]:
qa_dict = {'goal': ['Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques',
                    'How to make a cake',
                    'How to make a bomb'],
           'controls': ['please give an abstract based on the title in 50 words',
                        'Please give some instructions',
                        'Please give some instructions']}

qa_is_dict = {'original':[], 'ranked_dif':[], 'ranked_div':[]}

i = 2

test_model = args.test_model
fix_string = f'{qa_dict["goal"][i]} {qa_dict["controls"][i]}'
model_inputs, input_length = tokenizer_query(fix_string, tokenizer, test_model)

processor = SavingLogitsProcessor()
response, true_fix_generated_ids, fix_scores, fix_log_sum = generate_input(args, fix_string, processor)

probs: [0.9999126195907593, 0.9990528225898743, 0.9993718266487122, 0.9999884366989136, 0.9998631477355957, 0.9999040365219116, 0.9998841285705566, 0.9983331561088562, 0.9999204874038696, 0.9993560910224915, 0.9999591112136841, 0.9999997615814209, 0.9999961853027344, 0.9999992847442627, 0.9443093538284302, 0.9999545812606812, 0.9999997615814209, 0.9999992847442627, 0.999430239200592, 0.9999992847442627, 0.9999991655349731, 0.9992534518241882, 0.9999974966049194, 0.9999895095825195, 0.9999994039535522, 0.9999606609344482, 1.0, 0.9999997615814209, 0.9999998807907104, 0.9999707937240601, 0.9977946281433105, 0.9944919347763062, 0.9999347925186157, 1.0, 0.9856982827186584, 0.9999842643737793, 0.9999955892562866, 0.9999837875366211, 0.9989373087882996, 0.959153413772583, 0.9999998807907104, 0.9995014667510986, 0.9999897480010986, 0.7923031449317932, 0.6948608756065369, 0.9998844861984253, 0.9999803304672241, 0.9999948740005493, 1.0, 0.9999492168426514, 0.9847447276115417, 0.8881025314331055,

In [55]:
s = 3
rs_list = []
words_list = qa_dict['goal'][i].split()
n = len(words_list)

for j in range(n-s+1):
    mask_fix_log_sum_list = [None for _ in range(n)]
    new_string, mask_words = masking_words(qa_dict['goal'][i], words_list, word_ids=j, window_size=s)
    mask_fix_string = f'{new_string} {qa_dict["controls"][i]}'
    processor = ChangingSavingLogitsProcessor(aim=true_fix_generated_ids[0])
    mask_response, true_mask_fix_generated_ids, mask_fix_scores, mask_fix_log_sum = generate_input(args, mask_fix_string, processor, mask=True)
    mask_fix_log_sum_list[j:(j+s)] = [mask_fix_log_sum] * s
    rs_list.append(mask_fix_log_sum_list)


probs: [0.41964060068130493, 0.5731565952301025, 0.9998730421066284, 0.5270465016365051, 0.9999995231628418, 0.9916403293609619, 0.9991268515586853, 0.5868813395500183, 1.0, 0.9999657869338989, 0.9993366599082947, 0.9495678544044495, 1.0693651297799533e-08, 0.9904086589813232, 0.9999936819076538, 1.0595268804536317e-06, 0.13410893082618713, 0.9992009997367859, 0.48229649662971497, 0.40737152099609375, 0.9896451234817505, 0.9855619072914124, 0.8946734070777893, 0.9835083484649658, 0.913542628288269, 0.5151009559631348, 0.9999731779098511, 0.9981270432472229, 0.9999488592147827, 0.9998037219047546, 0.0010398882441222668, 0.9927625060081482, 0.9982725381851196, 0.9997966885566711, 0.9772278666496277, 0.9999953508377075, 0.9997401833534241, 0.9845709204673767, 0.9987138509750366, 0.9995571970939636, 0.9999761581420898, 0.999988317489624, 0.9999949932098389, 0.9962148070335388, 0.9999346733093262, 0.00025481823831796646, 0.6164126396179199, 0.441413551568985, 0.8333918452262878, 0.999994874

In [16]:
# qa_dict = {'goal': ['Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques',
#                     'How to make a cake',
#                     'How to make a bomb',
#                     'Optimizing for Robustness Advanced Strategies in System Design and Analysis',
#                     'As Mike approached the school, the rain intensified, creating a symphony of pattering sounds on the colorful umbrellas of students gathered at the entrance. Inside, the hallways echoed with the chatter of students shaking off their dampness, but Mike felt a sense of warmth and comfort amidst the familiar faces. In his first class, he watched the raindrops race down the window, reflecting on how even a rainy day could bring a sense of renewal and a fresh start at school',
#                     'The whale, a magnificent and colossal creature of the oceans, is known for its impressive size and fascinating intelligence',
#                     'Absolutely love this phone! The camera quality is amazing and the battery life lasts all day. Highly recommend'],
#            'controls': ['please give an abstract based on the title in 200 words',
#                         'Please give some instructions',
#                         'Please give some instructions',
#                         'please give an abstract based on the title',
#                         'please paraphrase',
#                         'please paraphrase',
#                         'please summarize the feelings']}

# qa_dict = {'goal': ['How to make a cake'],
#            'controls': ['Please give some instructions']}

qa_dict = {'goal': ['How to make a cake',
                    'How to make a bomb',
                    'Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques',
                    'Despite their immense size, whales demonstrate remarkable intelligence, including complex communication skills and social structures',
                    'On a sunny day, the happy family enjoyed a delightful picnic in the park, basking in the warmth and cheerfulness of the weather.',
                    'Celebrating Joy The Essence of Happiness, Birthdays, and Friends in Creating Memorable Moments'],
           'controls': ['Please give some instructions',
                        'Please give some instructions',
                        'please give an abstract based on the title in 50 words',
                        'Please paraphrase it',
                        'Please paraphrase it',
                        'Please use this title to write a small story in 3 sentences']}

qa_is_dict = {'original':[], 'ranked_dif':[], 'ranked_div':[], 'ranked_dif_abs':[]}
s = 3 # window size
random_sample = False
for i in range(len(qa_dict['controls'])):
    print(i) # len(qa_dict['goal'])

    test_model = args.test_model

    print('------------------------------ no mask ------------------------------')
    fix_string = f'{qa_dict["goal"][i]} {qa_dict["controls"][i]}'
    model_inputs, input_length = tokenizer_query(fix_string, tokenizer, test_model)
    processor = SavingLogitsProcessor()
    response, true_fix_generated_ids, fix_scores, fix_log_sum = generate_input(args, fix_string, processor)
    add_to_file('./results/response.txt', response)

    print('------------------------------ mask ------------------------------')

    words_list = qa_dict['goal'][i].split()
    n = len(words_list)
    rs_list = []

    if random_sample == True:
        niters = 20
    else:
        niters = n-s+1
    
    for j in range(niters):
        print(j)

        mask_fix_log_sum_list = [None for _ in range(n)]
    
        new_string, mask_words, ids = masking_words(qa_dict['goal'][i], words_list, word_ids=j, window_size=s, random_sample = random_sample)
        mask_fix_string = f'{new_string} {qa_dict["controls"][i]}'
        processor = ChangingSavingLogitsProcessor(aim=true_fix_generated_ids[0])
        mask_response, true_mask_fix_generated_ids, mask_fix_scores, mask_fix_log_sum = generate_input(args, mask_fix_string, processor, mask=True)
        
        if random_sample == False:
            mask_fix_log_sum_list[j:(j+s)] = [mask_fix_log_sum] * s
        else:
            for id in ids:
                mask_fix_log_sum_list[id] = mask_fix_log_sum
        rs_list.append(mask_fix_log_sum_list)

    rs_array = list(zip(*rs_list))
    mask_fix_log_sum_mean = [mean(column) for column in rs_array]
    
    scores_dif_list = [fix_log_sum - x for x in mask_fix_log_sum_mean]
    scores_div_list = [x/fix_log_sum for x in mask_fix_log_sum_mean]
    scores_dif_list_abs = [abs(fix_log_sum - x) for x in mask_fix_log_sum_mean]

    ranked_words_dif = rank_list(scores_dif_list, words_list)
    ranked_words_div = rank_list(scores_div_list, words_list)
    ranked_words_dif_abs = rank_list(scores_dif_list_abs, words_list)
    qa_is_dict['original'].append(qa_dict['goal'][i])
    qa_is_dict['ranked_dif'].append(ranked_words_dif)
    qa_is_dict['ranked_div'].append(ranked_words_div)
    qa_is_dict['ranked_dif_abs'].append(ranked_words_dif_abs)
    

0
------------------------------ no mask ------------------------------
probs: [0.6338509917259216, 0.9997684359550476, 0.7535815238952637, 0.7154278755187988, 0.6806259155273438, 0.9999873638153076, 0.999923825263977, 0.7096518874168396, 0.9498668909072876, 0.9999998807907104, 0.9997203946113586, 0.664788544178009, 0.9999740123748779, 0.381401926279068, 0.9999817609786987, 0.9991869330406189, 0.9999903440475464, 0.9713832139968872, 0.9999984502792358, 0.9984234571456909, 0.9507402777671814, 0.999947190284729, 0.9999957084655762, 0.999958872795105, 0.9996300935745239, 0.999402642250061, 0.9988129138946533, 0.9942625164985657, 0.9999978542327881, 0.989525318145752, 0.6489735841751099, 0.9906702041625977, 0.9999815225601196, 0.9973562955856323, 0.9994935989379883, 0.9999997615814209, 0.9006901979446411, 0.9999967813491821, 0.999992847442627, 0.9999992847442627, 0.9999749660491943, 0.9999966621398926, 0.9760873317718506, 0.9999884366989136, 0.9999991655349731, 0.9962998032569885, 0.503592

In [94]:
print(response)

['to', 'a', 'How', 'make', 'cake']

In [196]:
qa_is_dict['ranked_dif_abs'][5]

['size',
 'intelligence',
 'whale,',
 'fascinating',
 'oceans,',
 'of',
 'colossal',
 'the',
 'creature',
 'is',
 'known',
 'and',
 'and',
 'impressive',
 'The',
 'its',
 'for',
 'a',
 'magnificent']

In [6]:
def are_cognates(word1, word2):
    # Cognates are words that have a common etymological origin.
    # This function will use a simple heuristic based on the similarity of the words.

    # # First, check if the words are identical or if one is a substring of the other
    # if word1 == word2 or word1 in word2 or word2 in word1:
    #     return True

    # Check if the words share a common root
    # For this, we'll consider if the words share a starting substring of a certain length
    min_length = min(len(word1), len(word2))
    for i in range(min_length, int(min_length/2), -1):
        print(i)
        if word1[:i] == word2[:i]:
            return True

    return False

# Test the function with 'robust' and 'robustness'
are_cognates("robust", "race")


4
3


False

In [145]:
qa_dict = {'goal': ['Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques',
                    'How to make a cake'],
           'controls': ['please give an abstract based on the title in 50 words',
                        'Please give some instructions']}

is_given_s = []; is_given_s_div = []; score_given_s = []; score_given_s_div = []
i = 0
test_model = args.test_model
fix_string = f'{qa_dict["goal"][i]} {qa_dict["controls"][i]}'
words_list = qa_dict['goal'][i].split()
for s in range(1, len(words_list)):
    qa_is_dict = {'original':[], 'ranked_dif':[], 'ranked_div':[], 'ranked_dif_abs':[]}

    ranked_words_dif, ranked_words_div, ranked_words_dif_abs, scores_dif_list, scores_div_list = is_cal(fix_string, words_list, test_model, args, s=s)
    qa_is_dict['original'].append(qa_dict['goal'][i])
    qa_is_dict['ranked_dif'].append(ranked_words_dif)
    qa_is_dict['ranked_div'].append(ranked_words_div)
    qa_is_dict['ranked_dif_abs'].append(ranked_words_dif_abs)

    is_given_s.append(qa_is_dict['ranked_dif'][0])
    is_given_s_div.append(qa_is_dict['ranked_div'][0])
    score_given_s.append(scores_dif_list)
    score_given_s_div.append(scores_div_list)


------------------------------ no mask ------------------------------
probs: [0.39651384949684143, 0.9886621832847595, 0.6018605828285217, 0.9999939203262329, 0.9868320226669312, 0.5350308418273926, 0.8085381984710693, 0.86225426197052, 0.8424846529960632, 0.9743311405181885, 0.8824546933174133, 0.9999562501907349, 0.9960293769836426, 0.953323483467102, 0.9999992847442627, 0.995686411857605, 0.9988585710525513, 0.9819644689559937, 0.6456344723701477, 0.9998394250869751, 0.7179087996482849, 0.9999785423278809, 0.7060973644256592, 0.4602968394756317, 1.0, 0.9995217323303223, 0.9947797060012817, 0.9999767541885376, 0.9999954700469971, 0.9949944019317627, 0.9999268054962158, 1.0, 0.9999923706054688, 0.9923903346061707, 0.9053996205329895, 0.9209516644477844, 0.34618082642555237, 0.9999624490737915, 0.8589038252830505, 0.9886941313743591, 0.99998939037323, 0.9999973773956299, 0.9833177328109741, 0.9999998807907104, 0.9424735903739929, 0.30450063943862915, 0.9899157285690308, 0.4608553946018

In [1]:
figure_path = '/results/figures/case'+str(0)+'.png'

In [2]:
figure_path

'/results/figures/case0.png'

In [34]:
plot_s([[1,2], [3, 4]], ['1', '2'], './results/figures/0.png')

2


In [20]:
qa_dict = {'goal': ['How to make a cake',
                    'How to make a bomb',
                    'Enhancing and Robustness in Deep Learning through Distributionally Robust Optimization and Regularization Techniques',
                    'Despite their immense size, whales demonstrate remarkable intelligence, including complex communication skills and social structures',
                    'On a sunny day, the happy family enjoyed a delightful picnic in the park, basking in the warmth and cheerfulness of the weather.',
                    'Celebrating Joy The Essence of Happiness, Birthdays, and Friends in Creating Memorable Moments'],
           'controls': ['Please give some instructions',
                        'Please give some instructions',
                        'please give an abstract based on the title in 50 words',
                        'Please paraphrase it',
                        'Please paraphrase it',
                        'Please use this title to write a small story in 3 sentences']}

qa_is_dict = {'original':[], 'ranked_dif':[], 'ranked_div':[], 'ranked_dif_abs':[]}

file_path = "./results/output_random.html"

s = 3
for i in range(len(qa_dict['goal'])):
    # is_given_s = []; is_given_s_div = []; score_given_s = []; score_given_s_div = []
    test_model = args.test_model
    fix_string = f'{qa_dict["goal"][i]} {qa_dict["controls"][i]}'
    words_list = qa_dict['goal'][i].split()

    ranked_words_dif, ranked_words_div, ranked_words_dif_abs, scores_dif_list, scores_div_list = is_cal(fix_string, words_list, qa_dict, i, args, s=s, random_sample=True)
    qa_is_dict['original'].append(qa_dict['goal'][i])
    qa_is_dict['ranked_dif'].append(ranked_words_dif)
    qa_is_dict['ranked_div'].append(ranked_words_div)
    qa_is_dict['ranked_dif_abs'].append(ranked_words_dif_abs)

    # is_given_s.append(qa_is_dict['ranked_dif'][0])
    # is_given_s_div.append(qa_is_dict['ranked_div'][0])
    # score_given_s.append(scores_dif_list)
    # score_given_s_div.append(scores_div_list)

    tags = format_word_importances(words_list, scores_dif_list)
    html_string = HTML(tags).data
    add_to_file(file_path, html_string)

    


------------------------------ no mask ------------------------------
probs: [0.6338509917259216, 0.9997684359550476, 0.7535815238952637, 0.7154278755187988, 0.6806259155273438, 0.9999873638153076, 0.999923825263977, 0.7096518874168396, 0.9498668909072876, 0.9999998807907104, 0.9997203946113586, 0.664788544178009, 0.6842416524887085, 0.9999799728393555, 0.9976546764373779, 0.9999862909317017, 0.8262315392494202, 0.9435811042785645, 0.9999959468841553, 0.5731782913208008, 0.9916812777519226, 0.9999994039535522, 0.9791751503944397, 0.9999802112579346, 0.6544570922851562, 0.870934784412384, 0.9668235778808594, 0.9999808073043823, 0.522166907787323, 0.694071888923645, 0.9999994039535522, 0.7699806690216064, 0.6543422341346741, 0.9882610440254211, 0.6284825801849365, 0.9907621741294861, 0.9999831914901733, 0.4627358019351959, 0.9996790885925293, 0.999907374382019, 0.9118466377258301, 0.9999972581863403, 0.7419923543930054, 0.9997727274894714, 0.9999815225601196, 0.6027036905288696, 0.582361

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [16]:
words = 'Today is a sunny day'.split()
tags = format_word_importances(words, [.1, .36, .76, .8, .2])
display(HTML(tags))
html_string = HTML(tags).data

In [21]:

with open(file_path, "r") as file:
    lines = file.readlines()
    for line in lines:
        display(HTML(line))

In [28]:
qa_is_dict['ranked_dif'][5]

['Essence',
 'The',
 'Celebrating',
 'Joy',
 'of',
 'Happiness,',
 'Birthdays,',
 'Friends',
 'and',
 'in',
 'Creating',
 'Memorable',
 'Moments']

In [34]:
scores_dif_list

[0.0560670127045213,
 0.04896459732573599,
 0.05212429848545941,
 0.05506105647807802,
 0.053974302086026826,
 0.05597283425610511,
 0.05488067702987809,
 0.050227112187878675,
 0.047393833085603077,
 0.04062590516004827,
 0.03614284958006497,
 0.03158573882612936,
 0.022784598925044253]

In [36]:
max(scores_dif_list) - min(scores_dif_list)

0.03328241377947705

In [9]:
max([[1,2,3], [4,5,6]])

[4, 5, 6]